# Build clean dataset

Based on the previously performed EDA, we reduce our data to a common cleaned data set. This data set is then the basis for all further steps. To do this, the 4 individual data frames previously created for the EDA are loaded, merged, reduced to relevant columns and cleaned. The final data frame is again stored in the data folder.

In [41]:
import pandas as pd
import os
import numpy as np
import datetime
import decimal

## Load Data

In [42]:
df_satisfaction = pd.read_feather("../data/Data_Satisfaction")
df_meta = pd.read_feather("../data/Data_Metadata")
df_text = pd.read_feather("../data/Data_Unlabeled")
df_text_labeled = pd.read_feather("../data/Data_Labeled")

## Tranform and clean data

### Transform

In [43]:
df_satisfaction_min = df_satisfaction[df_satisfaction.FrageCode == 'wime_gesamtzuf'] #only keep overall satisfaction
df_satisfaction_min = df_satisfaction_min.drop(['FrageCode'], axis=1)
df_satisfaction_min = df_satisfaction_min.rename(columns={"Value": "wime_gesamtzuf"})

df_temp = df_text.merge(df_satisfaction_min, left_on='ParticipantId', right_on='ParticipantId',how='left')
df_full = df_temp.merge(df_meta, left_on='ParticipantId', right_on='ParticipantId',how='left')

df_full = df_full.drop(['Date_x'], axis=1)
df_full = df_full.drop(['Date_y'], axis=1)

# reorder colnames for export
new_cols = ['ParticipantId',
            'Date',
            'Value',
            'Language',
            'wime_gesamtzuf',
            'S_sex',
            'S_alter',
            'u_bezugsart',
            'u_fahrausweis',
            'u_ga',
            'u_klassencode',
            'u_preis',
            'u_ticket',
            'device_type',
            'dispcode',
            'fg_startort',
            'fg_abfahrt',
            'ft_startort',
            'ft_abfahrt',
            'ft_zielort',
            'ft_ankunft',
            'fg_zielort',
            'fg_ankunft',
            'ft_haltestellen',
            'ft_vm_kurz',
            'R_anschluss',
            'R_stoerung',
            'R_zweck',
            'S_berufstaetigkeit',
            'S_wohnsitz',
            'S_Usertyp1',
            'S_Usertyp2',
            'S_Usertyp3'
           ]

df_full = df_full[new_cols]


#Replace it with None values which can be interpreted by pandas
dates = ['Date','fg_abfahrt','ft_abfahrt','ft_ankunft','fg_ankunft']

for x in dates:
    df_full[[x]] = df_full[[x]].astype(object).where(df_full[[x]].notnull(), None)

### Cleaning

In [44]:
## Col specific cleaning - Fahrzweck

df_full.R_zweck = df_full.R_zweck.replace('Freizeitfahrt  ohne Übernachtung (Ausflug, Kino, Sport, Besuch, usw.)','Freizeitfahrt / Ferienreise oder alltägliche Erledigungen')
df_full.R_zweck = df_full.R_zweck.replace('Freizeitfahrt/ private Ferienreise/ alltägliche Erledigungen (z.B. Arztbesuch, Einkaufen, jmd. Abhol','Freizeitfahrt / Ferienreise oder alltägliche Erledigungen')
df_full.R_zweck = df_full.R_zweck.replace('Freizeitfahrt/ private Ferienreise\r\n','Freizeitfahrt / Ferienreise oder alltägliche Erledigungen')
df_full.R_zweck = df_full.R_zweck.replace('Alltägliche Erledigungen (z.B. Arztbesuch, Einkaufen, jmd. abholen)','Freizeitfahrt / Ferienreise oder alltägliche Erledigungen')
df_full.R_zweck = df_full.R_zweck.replace('Private Ferienreise (Reise mit mind. 1 Übernachtung)','Freizeitfahrt / Ferienreise oder alltägliche Erledigungen')
df_full.R_zweck = df_full.R_zweck.replace('alltägliche Erledigungen (z.B. Arztbesuch, Einkaufen, jmd. Abholen)\r\n','Freizeitfahrt / Ferienreise oder alltägliche Erledigungen')
df_full.R_zweck = df_full.R_zweck.replace('Fahrt zum Arbeitsort / Ausbildungsort','Pendeln')
df_full.R_zweck = df_full.R_zweck.replace('Fahrt zum Arbeitsort','Pendeln')
df_full.R_zweck = df_full.R_zweck.replace('Fahrt zum Ausbildungsort','Pendeln')
df_full.R_zweck = df_full.R_zweck.replace('Fahrt vom oder zum Arbeits-/ Ausbildungsort','Pendeln')

## Col specific cleaning - U_Bezugsart
df_full.u_bezugsart = df_full.u_bezugsart.replace('MobileTicket','Mobile Ticket')

## Col specific cleaning - U_Ticket
df_full.u_ticket = df_full.u_ticket.replace('3', np.nan)

## Col Language
df_full = df_full[df_full["Language"]=="Deutsch"].reset_index(drop=True) #Only keep german surveys

### Write dataframe to file

In [45]:
df_full.to_feather("../data/Data_Clean")